In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import pdb
import copy
from scipy import sparse
import anndata
import swan_vis as swan


p = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

In [2]:
h5 = '../cerberus_annot.h5'
ab = '../../talon/human_talon_abundance.tsv'
filt_ab = '../cerberus_filtered_abundance.tsv'
obs_col = 'sample'
min_tpm = 1
major_set = 'isos_sample_gene_90.tsv'
swan_file = 'swan.p'

In [3]:
mane_file = o = '/Users/fairliereese/mortazavi_lab/data/rnawg/refs/v40_gene_metadata.tsv'

In [4]:
# get list of gene inds that are >=1 tpm and 
# had a mane iso
inds = pd.read_csv('temp_gids_1_tpm.tsv', sep='\t')
inds = inds.gids.tolist()

mane = pd.read_csv(mane_file, sep='\t')
mane = mane.loc[mane.MANE_Select == True]
mane['gid'] = cerberus.get_stable_gid(mane, col='gid')
mane = mane.gid.tolist()

In [5]:
inds = list(set(inds)&(set(mane)))
print(len(inds))

17629


In [24]:
obs_col = 'sample'
sg = swan.read('swan.p')
gene_subset=inds

Read in graph from swan.p


In [45]:
# get the major tss, ic, tes, and isoform from each sample
adatas = [sg.adata, sg.tss_adata, sg.ic_adata, sg.tes_adata]
t_dfs = [sg.t_df, sg.tss_adata.var, sg.ic_adata.var, sg.tes_adata.var]
feats = ['triplet', 'tss', 'ic', 'tes']
id_cols = {'triplet': 'tid', 'tss': 'tss_id', 'ic': 'ic_id', 'tes': 'tes_id'}
major_df = pd.DataFrame()

for feat, adata, t_df in zip(feats, adatas, t_dfs):
    # if feat == 'triplet':
    #     continue
    print(feat)
    
    # get pi value for each feature in the specified gb category
    df, _ = swan.calc_pi(adata, t_df, obs_col=obs_col)
    
    df = df.sparse.to_dense()
    df = df.transpose()
    
    # pdb.set_trace()

    # merge with gene info
    id_col = id_cols[feat]
    print(t_df.head())
    t_df = t_df.copy(deep=True)
    if feat == 'triplet':
        drop = True
    else:
        drop = False
    t_df.reset_index(drop=drop,inplace=True)
    t_df = t_df[[id_col, 'gid']]
    df = df.merge(t_df, how='inner', on=id_col)
    df.set_index([id_col, 'gid'], inplace=True)
    # t_df = sg.t_df.copy(deep=True)
    # t_df.reset_index(drop=True, inplace=True)
    # t_df = t_df[['tid', 'gname', 'gid']]
    # df = df.merge(t_df, how='inner', on='tid')
    # df.set_index(['tid', 'gname', 'gid'], inplace=True)
    
    # pivot and coerce into lengthwise format
    df = df.melt(ignore_index=False, value_name='pi', var_name=obs_col)
    df = df.dropna(subset=['pi'])
    df.reset_index(inplace=True)
    
    # remove unexpressed isoforms
    df = df.loc[df.pi > 0]
    
    # limit to detected genes
    df['gid_stable'] = cerberus.get_stable_gid(df, 'gid')
    if gene_subset:
        df = df.loc[df.gid_stable.isin(gene_subset)]
        
    # sort by gene, sample, and pi value
    # dedupe across the gene and sample cols; take the top-expressed isoform
    df = df.sort_values(by=['gid', obs_col, 'pi'],
                        ascending=[False, False, False])

    df = df.drop_duplicates(subset=['gid', obs_col], keep='first')
    
    # rename some columns
    if feat == 'triplet': 
        df.rename({'pi': 'tid_pi'}, axis=1, inplace=True)
    else:
        pi_col = '{}_pi'.format(feat)
        df.rename({'pi': pi_col}, axis=1, inplace=True)
    
    # first entry
    if major_df.empty:
        major_df = df.copy(deep=True)
    else:
        major_df = major_df.merge(df, how='outer', on=['gid', 'sample', 'gid_stable'])        

triplet
                                tname              gid   gname  \
tid                                                              
ENSG00000000003[1,1,1]  TSPAN6[1,1,1]  ENSG00000000003  TSPAN6   
ENSG00000000003[1,1,5]  TSPAN6[1,1,5]  ENSG00000000003  TSPAN6   
ENSG00000000003[1,1,6]  TSPAN6[1,1,6]  ENSG00000000003  TSPAN6   
ENSG00000000003[1,1,7]  TSPAN6[1,1,7]  ENSG00000000003  TSPAN6   
ENSG00000000003[1,1,8]  TSPAN6[1,1,8]  ENSG00000000003  TSPAN6   

                                                                     path  \
tid                                                                         
ENSG00000000003[1,1,1]  [923954, 923955, 923956, 923957, 923958, 92395...   
ENSG00000000003[1,1,5]  [923954, 923955, 923956, 923957, 923958, 92395...   
ENSG00000000003[1,1,6]  [923954, 923955, 923956, 923957, 923958, 92395...   
ENSG00000000003[1,1,7]  [923954, 923955, 923956, 923957, 923958, 92395...   
ENSG00000000003[1,1,8]  [923954, 923955, 923956, 923957, 923958, 92

In [46]:
major_df.head()

,tid,gid,sample,tid_pi,gid_stable,tss_id,tss_pi,ic_id,ic_pi,tes_id,tes_pi
0,"ENSG00000285043[5,15,1]",ENSG00000285043,wtc11,95.815292,ENSG00000285043,ENSG00000285043_5,96.392494,ENSG00000285043_15,96.825394,ENSG00000285043_1,99.783554
1,"ENSG00000285043[5,15,1]",ENSG00000285043,vessels,92.857140,ENSG00000285043,ENSG00000285043_5,93.095238,ENSG00000285043_15,94.642860,ENSG00000285043_1,100.000000
2,"ENSG00000285043[5,15,1]",ENSG00000285043,pgp1_excite_neuron,81.538460,ENSG00000285043,ENSG00000285043_5,83.076927,ENSG00000285043_15,81.538460,ENSG00000285043_1,100.000000
3,"ENSG00000285043[5,15,1]",ENSG00000285043,pgp1_endo,80.000000,ENSG00000285043,ENSG00000285043_5,80.000000,ENSG00000285043_15,80.000000,ENSG00000285043_1,100.000000
4,"ENSG00000285043[5,15,1]",ENSG00000285043,pgp1_astro,85.310730,ENSG00000285043,ENSG00000285043_5,85.875710,ENSG00000285043_15,85.310730,ENSG00000285043_1,100.000000


In [34]:
df.rename({'pi': 'tss_pi'}, axis=1, inplace=True)

In [35]:
df.head()

,tss_id,gid,sample,tss_pi,gid_stable
1130785,ENSG00000285043_5,ENSG00000285043,wtc11,96.392494,ENSG00000285043
801550,ENSG00000285043_5,ENSG00000285043,vessels,93.095238,ENSG00000285043
1966772,ENSG00000285043_5,ENSG00000285043,pgp1_excite_neuron,83.076927,ENSG00000285043
1621859,ENSG00000285043_5,ENSG00000285043,pgp1_endo,80.000000,ENSG00000285043
536731,ENSG00000285043_5,ENSG00000285043,pgp1_astro,85.875710,ENSG00000285043


In [20]:
major_df_back = major_df.copy(deep=True)

In [32]:
major_df_back.head()

,tid,gname,gid,sample,tid_pi,gid_stable
4255556,"ENSG00000285043[5,15,1]",ENSG00000285043,ENSG00000285043,wtc11,95.815292,ENSG00000285043
3001527,"ENSG00000285043[5,15,1]",ENSG00000285043,ENSG00000285043,vessels,92.857140,ENSG00000285043
7517393,"ENSG00000285043[5,15,1]",ENSG00000285043,ENSG00000285043,pgp1_excite_neuron,81.538460,ENSG00000285043
6164079,"ENSG00000285043[5,15,1]",ENSG00000285043,ENSG00000285043,pgp1_endo,80.000000,ENSG00000285043
1974542,"ENSG00000285043[5,15,1]",ENSG00000285043,ENSG00000285043,pgp1_astro,85.310730,ENSG00000285043


In [36]:
major_df_back = major_df_back.merge(df, how='outer', on=['gid', 'sample', 'gid_stable'])

In [39]:
major_df_back = major_df_back[0]

In [40]:
major_df_back.head()

,tid,gname,gid,sample,tid_pi,gid_stable,tss_id,tss_pi
0,"ENSG00000285043[5,15,1]",ENSG00000285043,ENSG00000285043,wtc11,95.815292,ENSG00000285043,ENSG00000285043_5,96.392494
1,"ENSG00000285043[5,15,1]",ENSG00000285043,ENSG00000285043,vessels,92.857140,ENSG00000285043,ENSG00000285043_5,93.095238
2,"ENSG00000285043[5,15,1]",ENSG00000285043,ENSG00000285043,pgp1_excite_neuron,81.538460,ENSG00000285043,ENSG00000285043_5,83.076927
3,"ENSG00000285043[5,15,1]",ENSG00000285043,ENSG00000285043,pgp1_endo,80.000000,ENSG00000285043,ENSG00000285043_5,80.000000
4,"ENSG00000285043[5,15,1]",ENSG00000285043,ENSG00000285043,pgp1_astro,85.310730,ENSG00000285043,ENSG00000285043_5,85.875710


In [41]:
major_df_back.loc[major_df_back.tid.isnull()]

,tid,gname,gid,sample,tid_pi,gid_stable,tss_id,tss_pi


In [42]:
major_df_back.loc[major_df_back.tss_id.isnull()]

,tid,gname,gid,sample,tid_pi,gid_stable,tss_id,tss_pi
